In [71]:
import numpy as np
import pandas as pd

In [72]:
from sklearn.impute import SimpleImputer ## there are missing values in the dataset used therefore
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
df = pd.read_csv("covid_toy.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [74]:
df["cough"].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [75]:
df["city"].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

<u>Note</u></br></br>
Age - numerical</br>
Fever - numerical</br>
Gender -  nominal categorical</br>
City - nominal categorical</br>
Cough - ordinal categorical

Check missing values

In [76]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

### Train test split

In [77]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=["has_covid"]),
    df["has_covid"],
    test_size=0.2)

In [78]:
X_train

,age,gender,fever,cough,city
48,66,Male,99.0,Strong,Bangalore
83,17,Female,104.0,Mild,Kolkata
21,73,Male,98.0,Mild,Bangalore
5,84,Female,NaN,Mild,Bangalore
57,49,Female,99.0,Strong,Bangalore
...,...,...,...,...,...
47,18,Female,104.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
78,11,Male,100.0,Mild,Bangalore
63,10,Male,100.0,Mild,Bangalore


## Doing column transformations without sklearn class

### 1. Using imputer on fever to fill missing values

In [79]:
si = SimpleImputer()

X_train_fever = si.fit_transform(X_train[["fever"]])
X_test_fever = si.fit_transform(X_test[["fever"]])

# just recall these result will be a numpy array

X_train_fever.shape

(80, 1)

### 2. Ordinal encoding on cough

In [80]:
oe = OrdinalEncoder(categories=[["Mild", "Strong"]])

X_train_cough = oe.fit_transform(X_train[["cough"]])
X_test_cough = oe.fit_transform(X_test[["cough"]])

X_train_cough.shape

(80, 1)

### 3. OneHotEncoding on gender and city

In [81]:
ohe = OneHotEncoder(drop="first", sparse_output=False)

X_train_gender_city = ohe.fit_transform(X_train[["gender", "city"]])
X_test_gender_city = ohe.fit_transform(X_test[["gender", "city"]])

X_train_gender_city.shape

(80, 4)

### 4. Extracting age from the original dataset to then at last concatenate all of the created array

In [82]:
X_train_age = X_train.drop(columns=["gender", "fever", "cough", "city"]).values

X_test_age = X_test.drop(columns=["gender", "fever", "cough", "city"]).values

X_train_age.shape

(80, 1)

### 5. Concatenating all of these to create a new dataframe that will be having all column transformation applied

In [83]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)

X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)

X_train_transformed.shape

(80, 7)

In [85]:
X_train_transformed = pd.DataFrame(X_train_transformed)
X_train_transformed.head()

,0,1,2,3,4,5,6
0,66.0,99.000000,1.0,0.0,0.0,0.0,1.0
1,17.0,104.000000,0.0,0.0,1.0,0.0,0.0
2,73.0,98.000000,1.0,0.0,0.0,0.0,0.0
3,84.0,100.783784,0.0,0.0,0.0,0.0,0.0
4,49.0,99.000000,0.0,0.0,0.0,0.0,1.0


## Doing transformation using sklearn ColumnTransformer

In [ ]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers=[
    ("tnf1", SimpleImputer(), ["fever"]), # for imputer
    ("tnf2", OrdinalEncoder(categories=[["Mild", "Strong"]]), ["cough"]), # for ordinal encoder
    ("tnf3", OneHotEncoder(drop="first", sparse_output=False), ["gender", "city"])  # for onehotencoder
], remainder="passthrough") # to retain other columns

In [92]:
transformer.fit_transform(X_train).shape

(80, 7)